In [5]:
import matplotlib
print(matplotlib.matplotlib_fname())
matplotlib.use('Agg')

/suphys/aloe8475/anaconda3/envs/phd/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc


In [2]:
#importing libraries 

import csv
import pandas 
import numpy as np
from tkinter import*
from tkinter import filedialog
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook
from bokeh.models import ColumnDataSource
from bokeh.layouts import column,row,gridplot,layout
from bokeh.models.widgets import Panel,Tabs
from bokeh.models import LinearAxis,Range1d
from IPython.core.debugger import set_trace
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from bokeh.palettes import all_palettes
output_notebook()


global downsample_rate
downsample_rate=10

Loading BokehJS ...

In [3]:
#opening files and transform to pandas function definitions

def open_dialog():
    # Create Tk root
    root = Tk()
    # Hide the main window
    root.withdraw()
    root.call('wm', 'attributes', '.', '-topmost', True)

  
    infiles = filedialog.askopenfilename(multiple=True)
    print('Number of files:','  ',len(infiles))
    return infiles

def HeaderReader(csVReader):
    #read csv file header
    headlen=4
    lin=[]
    for i in range(headlen-1):
        line=next(csVReader)
        lin.append(line)
    return dict(zip(lin[0],lin[2]))

def ChannelReader(channelstring):
    #prepare metadata
    columns=channelstring['channel_guide'].split(sep=',')
    chans=list()
    for col in columns:
        if col.find('/a')!=(-1):
            col=col[(col.find('/a')+1):]
        chans.append(col)
    return chans
def process_file(pandata,header):
    """
    processing files of n_back according to current file formatting
    There is a marker between intervals (number 999). The next two rows of data
    contain information about which pattern was in input(4 electrode)/output(2 electrode) 
    and whether it was train or test interval.
    Example: 
        out1 out2
    idx
   100     999  999  -> 999 is marker of previous interval end
   101     5    0    -> 5 is input pattern (0101 in binary, ie. two electrodes opened). 0 is train interval (1 would be test)
   102     1    -1   -> 1 is output pattern (01 in binary, ie. one output electrode being trained). -1 does not have meaning.
    
    
    
    """
    sel=list(pandata['out1'][pandata['out1']==999].index)
    sel2=sel.copy()
    sel.insert(0,-3)
    sel.pop(-1)
    
    index_start=list()
    index_end=list()
    interval=list()
    pat=list()
    outpat=list()
    descript=list()
    Vout1=list()
    Vout2=list()
    
    TrainOrTest=lambda a:'Train' if a==0 else 'Test'
    bin_str= lambda a: bin(int(a))[2:] if bin(int(a))[2:][-1]=='0' else '0'+bin(int(a))[2:]
    
    for i1,i2 in zip(sel,sel2):
        index_start.append(i1+3)
        index_end.append(i2)
        interval.append(TrainOrTest(pandata.iloc[i2+1,1]))
        pat.append(pandata.iloc[i2+1,0])
        outpat.append(pandata.iloc[i2+2,0])
        descript.append(TrainOrTest(pandata.iloc[i2+1,1])+'_IN:'+bin_str(pat[-1])+'__OUT:'+bin_str(outpat[-1]))
        vo1=np.array(pandata['out1'][i1+3:i2])
        extra_v1=np.random.choice(vo1[-10:],3) #this is just for index matching w. time. does not impact results.
        vo2=np.array(pandata['out2'][i1+3:i2])
        extra_v2=np.random.choice(vo2[-10:],3)
        Vout1.extend(list(np.concatenate((vo1,extra_v1))))
        Vout2.extend(list(np.concatenate((vo2,extra_v2))))
    
    index_start=pandas.DataFrame(index_start)
    index_end=pandas.DataFrame(index_end)
    interval=pandas.DataFrame(interval)
    pat=pandas.DataFrame(pat)
    outpat=pandas.DataFrame(outpat)
    descript=pandas.DataFrame(descript)
    Vout1=pandas.DataFrame(Vout1)
    Vout2=pandas.DataFrame(Vout2)
    Time=pandas.DataFrame(CreateTimeArray(pandata['out1'],header,downsample=downsample_rate))
   
    pdout=pandas.concat([pandata,Time,Vout1,Vout2,index_start,index_end,interval,pat,outpat,descript]
                     ,axis=1,ignore_index=True)
    
    pdout.columns=['out1','out2','Time','Vout1','Vout2','index_start','index_end','interval','pat',
                  'outpat','descript']
    return pdout
    
def CreateTimeArray(pandata,header=[],downsample=1,rate=1):
    #create time array from sampled acquisition
    tlen=pandata.shape[0]
    if header:
       
        rate=np.divide(1,int(header['Rate'])/downsample)
        
    else:
        rate=np.divide(1,int(rate)/downsample)
    
    return np.linspace(0,rate*tlen,tlen)
def open_and_transform():
    #file dialog for chosing files and transform data to pandas
    datafiles=list()
    metafiles=list()
    filenames=list()

    infiles=open_dialog()
    for fpath in infiles:
       
        with open(fpath) as csvDataFile:
            csvReader=csv.reader(csvDataFile)
            header=HeaderReader(csvReader)

        #chan_names=ChannelReader(header)
        chan_names=['out1','out2']
        df=pandas.read_csv(fpath,delimiter=';',skiprows=4,names=chan_names)
        index0=df['out1'][df['out1']==0].index[0:]
        df=df.drop(index0)
        df['out1']
        #print('channels:',chan_names)
        '''
        Create time array on timedata series. 
        '''
        #ndf=CreateTimeArray(header,df)
        #ndf.columns
        datafiles.append(df)
        metafiles.append(header)
        filenames.append(fpath[fpath.rfind('/')+1:])
        print(fpath)
    return datafiles,metafiles,filenames





def plot_multiple(pandata,header,indexes=0,pattern='all',interval='both'):
    cPat=lambda pat: pandata['pat'].dropna().unique() if pat=='all' else [pat]
    cInt=lambda inter: ['Train','Test'] if inter=='both' else [inter]
    if (indexes==0):
        slice_ind=pandata.loc[:,['index_start','index_end']][(pandata['pat'].isin(cPat(pattern))) & (pandata['interval'].isin(cInt(interval)))]
    else:
        pandataS=pandata.loc[indexes]
        slice_ind=pandataS.loc[:,['index_start','index_end']][(pandataS['pat'].isin(cPat(pattern))) & (pandataS['interval'].isin(cInt(interval)))]
   

    
    fg=figure(title='Current(V) vs. Time(s)',x_axis_label='Time(s)',y_axis_label='Current(V)')
    fg.xgrid.grid_line_color='gray'
    fg.ygrid.grid_line_color='gray'
    fg.xgrid.grid_line_alpha=0.5
    fg.ygrid.grid_line_alpha=0.5
    fg.outline_line_alpha=0.8
    fg.outline_line_color='black'
    
    
    for idx,row in slice_ind.iterrows():
            i1=row['index_start']
            i2=row['index_end']
            y1=pandata.loc[i1:i2-1,'out1']
            y2=pandata.loc[i1:i2-1,'out2']
            x=CreateTimeArray(y1,header,downsample_rate)
            fg.line(x,y1,color='blue',legend_label='Chanel1')
            fg.line(x,y2,color='red',legend_label='Chanel2')
    return fg

def plot_special(pandata,header):
    sliceT=pandata.loc[:,['index_start','index_end']][pandata['interval']=='Test']
    target=pandata.loc[:,['outpat']][pandata['interval']=='Test']
    
    checkAcc=lambda result,target: 1 if result==target else 0
    
    #plot whole time series
   
    y1=pandata['Vout1']
    y2=pandata['Vout2']
    tot_max=max(max(y1),max(y2))
    tot_min=min(min(y1),min(y2))
    y_range=Range1d(start=-0.01,end=tot_max+0.1*tot_max)
    fg=figure(title='Current(v) Time(s) w. accuracy',x_axis_label='Time(s)',y_axis_label='Current(V)',y_range=y_range)
    t=CreateTimeArray(y1,header,downsample_rate)
    fg.line(t,y1,color='blue',legend_label='Chanel1')
    fg.line(t,y2,color='red',legend_label='Chanel2')
    
    
    #plot scatter dots accuracy
    x_ac1=list()
    y_ac1=list()
    
    x_ac2=list()
    y_ac2=list()
    for idx,row in sliceT.iterrows():
        testing_pat=pandata.loc[idx,'pat']

        i1=row['index_start']
        i2=row['index_end']
        y1=pandata.loc[i1:i2-1,'out1']
        y2=pandata.loc[i1:i2-1,'out2']

        av=[y1.mean(),y2.mean()]
        booled=[int(el/max(av)) for el in av]
        tested_pat=bool_to_int(booled)

        target=int(pandata['outpat'][(pandata['interval']=='Train') & (pandata['pat']==testing_pat)].iloc[0])
        #print(av)
        #print(booled)
        #print(tested_pat)
        #print(target)
        if target==1:
            x_ac1.append(pandata.loc[i1,'Time'])
            y_ac1.append(checkAcc(tested_pat,target))
        elif target==2:
            x_ac2.append(pandata.loc[i1,'Time'])
            y_ac2.append(checkAcc(tested_pat,target))
            
    comb=y_ac1+y_ac2
    perc=sum(comb)/len(comb)
    print('Percentage of correct :')
    print('-----------------------')
    print(perc)
    print('.......................')
    fg.extra_y_ranges={'acc':Range1d(start=-0.01,end=1.2)}
    fg.add_layout(LinearAxis(y_range_name='acc',axis_label='Accuracy'),'right')
    fg.scatter(x_ac1,y_ac1,color='blue',y_range_name='acc',size=20)
    fg.scatter(x_ac2,y_ac2,color='red',y_range_name='acc',size=20)
    return fg
    
def select_and_plot(datafiles,name='net1'):
    #--------------------------------------
    selector=list(range(len(datafiles))) #select all files by default

    sel_dat=[datafiles[i] for i in selector]

    chan_names=[list(sel_dat[i]) for i in selector]

    #--------------------------------------
    li_dat=average_phase(sel_dat,chan_names)
    plot=create_bokeh_li_layout(li_dat,name=name)
    show(plot)
    return 

def bool_to_int(bool_arr):
    numb=0

    for idx,el in enumerate(bool_arr):
        numb=2**(idx)*el+numb
    return numb
def int_to_bool_array(int_pat):
        pat = [int(d) for d in str(bin(int(int_pat)))[2:]]
        pat.reverse()
        if len(pat)<9:
            pad=[0 for i in range(9-len(pat))]
            pat.extend(pad)
        # print(pat)
        return np.array(pat)










In [4]:
datafiles,metafiles,filenames=open_and_transform()


TclError: couldn't connect to display "localhost:10.0"

In [12]:
datafiles,metafiles,filenames=open_and_transform()




#print('Header')

#print('Order of measure')
#print('----------------')
#print(df['descript'].dropna())
#print('---------------')
#print(df.head())

# Patterns trained in this sample
downsample_rate=10
#print(df['pat'].dropna().unique())


for raw_file,meta,name in zip(datafiles,metafiles,filenames):
    print('\n\n\n\n')
    print('FileName:')
    print('---------')
    print(name)

    print('---------')
    print('Patterns: ',meta['PatList'])
    print('TrainVolt: ',meta['TrainVolt'])
    print('TestVolt:  ',meta['TestVolt'])
    print('GradSetpoint',meta['GradSetpoint'])
    print('VControlMax',meta['VControlMax'])
    print('PatThreshold',meta['PatThresh'])
    print('---------')
    print('---------')

    
    df=process_file(raw_file,meta)
    
    
    #figure 1. spliting curves w. different conditions. 
   
    fig1=plot_multiple(df,meta,indexes=0,interval='Train')
    #fig1=plot_multiple(df,meta,indexes=0,interval='Test') #plot testing curves
    fig1.plot_width=950
    fig1.plot_height=650
    show(fig1)
    
    l1=[*range(5,10,1)] #you can select different indexes changing the range
    fig2=plot_multiple(df,meta,indexes=l1,interval='Train')
    fig2.plot_width=950
    fig2.plot_height=650
    show(fig2)
    

    #figure 2. showing whole n-back w. 2 output electrodes results and accuracy. 


    fig3=plot_special(df,meta)
    fig3.plot_width=950
    fig3.plot_height=650
    show(fig3)



TclError: couldn't connect to display "localhost:10.0"